# 모델링 구현

## CountVectorizer를 활용한 벡터화

In [2]:
import pandas as pd
import numpy as np

DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

reviews = list(train_data['review'])
y = np.array(train_data['sentiment'])

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word', max_features=5000)

train_data_features = vectorizer.fit_transform(reviews)

In [4]:
train_data_features

<25000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1975048 stored elements in Compressed Sparse Row format>

## 학습 데이터와 검증 데이터 분리